# imports & setup

In [ ]:
import logging
import os
from pprint import pprint

import dask.dataframe as dd
import helpers
import numpy as np
import pandas as pd
import scipy.stats
from cloudpathlib import AnyPath as Path

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
handler = logging.StreamHandler()
formatter = logging.Formatter(
    "%(asctime)s %(process)d/%(threadName)s %(name)s %(levelname)s\n%(message)s"
)
handler.setFormatter(formatter)
logging.getLogger().handlers = [handler]

In [ ]:
logging.getLogger("gcsfs").setLevel("DEBUG")
logging.getLogger("google.cloud.bigquery").setLevel("DEBUG")
logging.getLogger("helpers").setLevel("DEBUG")
logging.getLogger("pandas").setLevel("DEBUG")
logging.getLogger("pyarrow").setLevel("DEBUG")

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")
logger.debug("test debug-level message")

# loading data

## metadata

### TCGA SKCM fractions (estimated)

In [ ]:
df_tcga_skcm_fractions_from_csx = helpers.datasets.load_tcga_skcm_fractions_from_csx()

### TCGA SKCM sample types (metastatic, primary, etc)

In [ ]:
from google.cloud import bigquery

bqclient = bigquery.Client()

query_string = """
SELECT * 
FROM `isb-cgc-bq.TCGA.biospecimen_gdc_current`
where project_short_name = "TCGA-SKCM"
    and sample_type_name = "Metastatic"
order by sample_barcode
"""

df_tcga_sample_metadata = (
    bqclient.query(query_string).result().to_dataframe(progress_bar_type="tqdm")
)

## RNA-seq TPM

### TCGA SKCM (real) bulk rna-seq

In [ ]:
uri_tcga_skcm_bulk_rnaseq = (
    "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/"
    "mixtures_real_tcga_skcm/tpm.parquet"
)

In [ ]:
!gsutil ls -lh $uri_tcga_skcm_bulk_rnaseq

In [ ]:
ddf_tcga_skcm_bulk_rnaseq = dd.read_parquet(uri_tcga_skcm_bulk_rnaseq, engine="pyarrow")
logger.debug(ddf_tcga_skcm_bulk_rnaseq.dtypes)

In [ ]:
ddf_tcga_skcm_bulk_rnaseq

### pseudo bulk rna-seq

In [ ]:
uri_pseudobulk_rnaseq = (
    "gs://liulab/data/pseudobulk_optimization/3_with_tcga_qc/"
    "mixtures/n_cells=5/malignant_from_one_sample=True/data.parquet"
)

In [ ]:
!gsutil ls -lh $uri_pseudobulk_rnaseq

In [ ]:
ddf_pseudobulk_rnaseq = (
    dd.read_parquet(
        uri_pseudobulk_rnaseq,
        engine="pyarrow",
    )
    .replace({"malignant_from_one_sample": {"True": True, "False": False}})
    .astype({"n_cells": "uint8", "malignant_from_one_sample": "bool"})
    .rename(columns={"tcga_aliquot_barcode_for_fractions": "aliquot_barcode"})
)

logger.debug(ddf_pseudobulk_rnaseq.dtypes)

In [ ]:
ddf_pseudobulk_rnaseq

# analysis

## determine immune high and low

In [ ]:
def make_labels_for_aliquots(df_cell_type_fractions, df_sample_metadata):
    immune_cell_types = ["B", "Macrophage", "NK", "T", "T CD4", "T CD8"]
    df_immune_fraction = (
        df_cell_type_fractions[immune_cell_types]
        .sum(axis="columns")
        .to_frame("immune_fraction")
        .rename_axis(index="aliquot_barcode")
        .reset_index()
        .assign(sample_barcode=lambda row: row["aliquot_barcode"].str[:-12])
    )
    return df_sample_metadata[["sample_barcode", "sample_type_name"]].merge(
        df_immune_fraction,
        left_on="sample_barcode",
        right_on="sample_barcode",
        validate="one_to_one",
    )


df_sample_metadata = make_labels_for_aliquots(
    df_tcga_skcm_fractions_from_csx, df_tcga_sample_metadata
)

In [ ]:
def add_immune_group_columns(df_sample_metadata):
    quintiles = pd.qcut(df_sample_metadata["immune_fraction"], 5, labels=False)
    df_sample_metadata = df_sample_metadata.assign(immune_low=quintiles == 0)
    df_sample_metadata = df_sample_metadata.assign(immune_high=quintiles == 4)
    return df_sample_metadata


df_sample_metadata = add_immune_group_columns(
    make_labels_for_aliquots(df_tcga_skcm_fractions_from_csx, df_tcga_sample_metadata)
)
df_sample_metadata

In [ ]:
df_sample_metadata[["immune_low", "immune_high"]].value_counts()

## more

### analysis - computing stats on each dataset individually

In [ ]:
import importlib

In [ ]:
importlib.reload(helpers.pseudobulk_evaluation.deg_analysis)
from helpers.pseudobulk_evaluation.deg_analysis import compute_stats_single_gene

In [ ]:
def add_immune_groups_to_rnaseq(ddf_bulk_rnaseq, df_sample_metadata):
    ddf = ddf_bulk_rnaseq.join(
        df_sample_metadata.set_index("aliquot_barcode")[
            ["immune_fraction", "immune_low", "immune_high"]
        ]
    )
    return ddf.groupby("gene_symbol").apply(compute_stats_single_gene)


gene_stats = add_immune_groups_to_rnaseq(ddf_tcga_skcm_bulk_rnaseq, df_sample_metadata)

### analysis - combining all data, then computing stats

In [ ]:
# merge real and pseudo data
ddf_bulk_rnaseq_all = ddf_tcga_skcm_bulk_rnaseq.merge(
    ddf_pseudobulk_rnaseq,
    on=["aliquot_barcode", "gene_symbol"],
    how="inner",
    suffixes=["_tcga_skcm", "_pseudo"],
)

In [ ]:
ddf_bulk_rnaseq_all

### join these fractions onto bulk rna-seq data


In [ ]:
ddf_bulk_rnaseq_all_with_immune = ddf_bulk_rnaseq_all.merge(
    df_immune_fraction_by_aliquot_barcode,
    left_on="aliquot_barcode",
    right_on="sample_id",
)

ddf_bulk_rnaseq_all_with_immune

In [ ]:
# compute immune high and low
immune_threshold_low = ddf_bulk_rnaseq_all_with_immune["immune_fraction"].quantile(0.2)
immune_threshold_high = ddf_bulk_rnaseq_all_with_immune["immune_fraction"].quantile(0.8)

In [ ]:
immune_threshold_low, immune_threshold_high

In [ ]:
ddf_bulk_rnaseq_all_with_immune["immune_low"] = (
    ddf_bulk_rnaseq_all_with_immune["immune_fraction"] <= immune_threshold_low
)

ddf_bulk_rnaseq_all_with_immune["immune_high"] = (
    ddf_bulk_rnaseq_all_with_immune["immune_fraction"] >= immune_threshold_high
)

In [ ]:
df_bulk_rnaseq_all_with_immune = ddf_bulk_rnaseq_all_with_immune.compute()

In [ ]:
df_bulk_rnaseq_all_with_immune["aliquot_barcode"].value_counts()

In [ ]:
df_bulk_rnaseq_all_with_immune[["immune_low", "immune_high"]].value_counts()

#### compute stats for each gene

In [ ]:
### compute p-values...


def compute_stats(df):
    immune_low = df[df["immune_low"]]
    immune_high = df[df["immune_high"]]
    pval_pseudo = scipy.stats.mannwhitneyu(
        immune_high["tpm_pseudo"].values, immune_low["tpm_pseudo"].values
    )[1]
    neglog10pval_pseudo = -np.log10(pval_pseudo)
    foldchange_pseudo = (
        immune_high["tpm_pseudo"].mean() / immune_low["tpm_pseudo"].mean()
    )
    log2foldchange_pseudo = np.log2(foldchange_pseudo)

    pval_real = scipy.stats.mannwhitneyu(
        immune_high["tpm_tcga_skcm"].values, immune_low["tpm_tcga_skcm"].values
    )[1]
    neglog10pval_real = -np.log10(pval_real)
    foldchange_real = (
        immune_high["tpm_tcga_skcm"].mean() / immune_low["tpm_tcga_skcm"].mean()
    )
    log2foldchange_real = np.log2(foldchange_real)

    return pd.Series(
        dict(
            pval_pseudo=pval_pseudo,
            foldchange_pseudo=foldchange_pseudo,
            log2foldchange_pseudo=log2foldchange_pseudo,
            neglog10pval_pseudo=neglog10pval_pseudo,
            signedneglog10pval_pseudo=(
                neglog10pval_pseudo * np.sign(log2foldchange_pseudo)
            ),
            pval_real=pval_real,
            foldchange_real=foldchange_real,
            log2foldchange_real=log2foldchange_real,
            neglog10pval_real=neglog10pval_real,
            signedneglog10pval_real=(neglog10pval_real * np.sign(log2foldchange_real)),
        )
    )


df_gene_stats_by_immune = df_bulk_rnaseq_all_with_immune.groupby("gene_symbol").apply(
    compute_stats
)
df_gene_stats_by_immune = df_gene_stats_by_immune.reset_index()

In [ ]:
df_gene_stats_by_immune

In [ ]:
# how many genes have valid stats?

logger.debug(df_gene_stats_by_immune["pval_pseudo"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["foldchange_pseudo"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["pval_real"].isna().value_counts())
logger.debug(df_gene_stats_by_immune["foldchange_real"].isna().value_counts())

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_real",
    y="neglog10pval_real",
    title="real (tcga skcm): immune high / low",
    hover_name="gene_symbol",
    hover_data=["foldchange_real", "pval_real"],
)
fig.update_xaxes(range=(-10, 10))
fig.update_yaxes(range=(0, 30))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=1000, height=500)

In [ ]:
fig.show(renderer="browser")

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_pseudo",
    y="neglog10pval_pseudo",
    title="pseudobulks: immune high / low",
    hover_name="gene_symbol",
    hover_data=["foldchange_pseudo", "pval_pseudo"],
)
fig.update_xaxes(range=(-10, 10))
fig.update_yaxes(range=(0, 30))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=2, width=800, height=600)

In [ ]:
fig.show(renderer="browser")

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="signedneglog10pval_real",
    y="signedneglog10pval_pseudo",
    trendline="ols",
    title="signed -log10(p-values): pseudobulks vs tcga skcm",
    hover_name="gene_symbol",
    hover_data=["log2foldchange_real", "log2foldchange_pseudo"],
)
fig.update_xaxes(range=(-25, 25))
fig.update_yaxes(range=(-25, 25))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=500, height=500)

In [ ]:
fig.show(renderer="browser")

In [ ]:
df_gene_stats_by_immune.info()

In [ ]:
fig = px.scatter(
    df_gene_stats_by_immune,
    x="log2foldchange_real",
    y="log2foldchange_pseudo",
    trendline="ols",
    title="logs(fold-change): pseudobulks vs tcga skcm",
    hover_name="gene_symbol",
    # hover_data=["log2foldchange_real", "log2foldchange_pseudo"],
)
# fig.update_xaxes(range=(-25, 25))
# fig.update_yaxes(range=(-25, 25))
fig.update_traces(marker=dict(size=3))
fig.show(renderer="png", scale=1, width=500, height=500)
fig.show(renderer="browser")

In [ ]:
fig.show(renderer="browser")

#### what's the overlap for significant genes between real, pseudo?

In [ ]:
df_gene_stats_by_immune["percentile_neglog10pval_pseudo"] = df_gene_stats_by_immune[
    "neglog10pval_pseudo"
].rank(pct=True)
df_gene_stats_by_immune["percentile_neglog10pval_real"] = df_gene_stats_by_immune[
    "neglog10pval_real"
].rank(pct=True)

In [ ]:
THRESHOLD = 0.9

df_gene_stats_by_immune["top_pseudo"] = (
    df_gene_stats_by_immune["percentile_neglog10pval_pseudo"] > THRESHOLD
)
df_gene_stats_by_immune["top_real"] = (
    df_gene_stats_by_immune["percentile_neglog10pval_real"] > THRESHOLD
)
df_gene_stats_by_immune["top_both"] = (
    df_gene_stats_by_immune["top_pseudo"] & df_gene_stats_by_immune["top_real"]
)
pd.crosstab(df_gene_stats_by_immune["top_pseudo"], df_gene_stats_by_immune["top_real"])

In [ ]:
# TODO - a good summary stat here would be the odds ratio of this contingency table

In [ ]:
# TODO - benjamini hochberg for a FDR-corrected significance test (use FDR of 0.1)

In [ ]:
# TODO - also do scatter for fold change

In [ ]:
# is it the same genes that showed up in the PCA analysis?